1. building the model

In [1]:
%pip install langchain_openai langchain_community langchain psycopg2 chromadb -q


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import langchain_openai
import langchain_community
import langchain
import psycopg2  
import chromadb
import pydantic

In [32]:
# #Lange versie zonder hulp van lanchain

# def connect_postgresql():
#     # Open database connection
#     # Connect to the database
#     db = psycopg2.connect(
#         "dbname=BIRD user=postgres host=localhost password=root port=5432"
#     )
#     return db


# def execute_postgresql_query(cursor, query):
#     """Execute a PostgreSQL query."""
#     cursor.execute(query)
#     result = cursor.fetchall()
#     return result


# def perform_query_on_postgresql_databases(query):
#     db = connect_postgresql()
#     cursor = db.cursor()
#     result = execute_postgresql_query(cursor, query)
#     db.close()
#     return result


# def print_postgresql_dialect():
#     # Get database version (Dialect)
#     query = "SELECT version();"
#     result = perform_query_on_postgresql_databases(query)
#     print("Database Dialect / Version: ", result[0][0])


# def print_postgresql_table_names():
#     # Get table names
#     query = """
#         SELECT table_name
#         FROM information_schema.tables
#         WHERE table_schema = 'public';
#     """
#     result = perform_query_on_postgresql_databases(query)
#     print("Table Names:")
#     for table in result:
#         print(table[0])


# def print_postgresql_table_info(table_name):
#     # Get table info (columns and types)
#     query = f"""
#         SELECT column_name, data_type
#         FROM information_schema.columns
#         WHERE table_name = '{table_name}';
#     """
#     result = perform_query_on_postgresql_databases(query)
#     print(f"Table Info for {table_name}:")
#     for column in result:
#         print(f"Column: {column[0]}, Type: {column[1]}")


# # Example usage
# print_postgresql_dialect()  # Prints database version
# print_postgresql_table_names()  # Prints table names
# print_postgresql_table_info("your_table_name")  # Replace 'your_table_name' with an actual table name


In [3]:
#Korte versie met lanchain hulp
from langchain_community.utilities.sql_database import SQLDatabase
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Access variables using os.getenv()
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")



db = SQLDatabase.from_uri(f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}")

In [4]:
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

postgresql
['account', 'alignment', 'atom', 'attendance', 'attribute', 'badges', 'bond', 'budget', 'card', 'cards', 'circuits', 'client', 'colour', 'comments', 'connected', 'constructorresults', 'constructors', 'constructorstandings', 'country', 'customers', 'disp', 'district', 'drivers', 'driverstandings', 'event', 'examination', 'expense', 'foreign_data', 'frpm', 'gasstations', 'gender', 'hero_attribute', 'hero_power', 'income', 'laboratory', 'laptimes', 'league', 'legalities', 'loan', 'major', 'match', 'member', 'molecule', 'order', 'patient', 'pitstops', 'player', 'player_attributes', 'posthistory', 'postlinks', 'posts', 'products', 'publisher', 'qualifying', 'race', 'races', 'results', 'rulings', 'satscores', 'schools', 'seasons', 'set_translations', 'sets', 'status', 'superhero', 'superpower', 'tags', 'team', 'team_attributes', 'trans', 'transactions_1k', 'users', 'votes', 'yearmonth', 'zip_code']

CREATE TABLE "order" (
	order_id BIGINT DEFAULT '0'::bigint NOT NULL, 
	account_id

In [5]:
openai_api_key = os.getenv("OPENAI_API_KEY")
lanchain_tracking_v2 = os.getenv("LANCHAIN_TRACKING_V2")
langchain_api_key = os.getenv("LANCHAIN_API_KEY")

In [7]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


#voor pydantic
class Sql (BaseModel):
  sql: str

llm = ChatOpenAI(model="gpt-4o", temperature=0)

generate_query = create_sql_query_chain(llm,db)
#easy voorbeeld
query= generate_query.invoke({"question" : "What is the ratio of customers who pay in EUR against customers who pay in CZK?"})
print(f"easy Query: \n\n{query}")
#moderate voorbeeld
# query= generate_query.invoke({"question" : "In 2012, who had the least consumption in LAM?"})
# print(f"Moderate Query: \n\n{query}")
# #hard voorbeeld
# query= generate_query.invoke({"question" : "What is the difference in the annual average consumption of the customers with the least amount of consumption paid in CZK for 2013 between SME and LAM, LAM and KAM, and KAM and SME?"})
# print(f"Challenging Query: \n\n{query}")

easy Query: 

```sql
SELECT 
    (SELECT COUNT(*) FROM "customers" WHERE "currency" = 'EUR')::float / 
    (SELECT COUNT(*) FROM "customers" WHERE "currency" = 'CZK')::float AS eur_to_czk_ratio;
```


In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)

'Error: (psycopg2.errors.SyntaxError) syntax error at or near "```"\nLINE 1: ```sql\n        ^\n\n[SQL: ```sql\nSELECT \n    COUNT(CASE WHEN "currency" = \'EUR\' THEN 1 END) AS eur_count,\n    COUNT(CASE WHEN "currency" = \'CZK\' THEN 1 END) AS czk_count\nFROM \n    "customers";\n```]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

In [ ]:
chain = generate_query | execute_query
chain.invoke({"question": "What is the ratio of customers who pay in EUR against customers who pay in CZK?"})


'Error: (psycopg2.errors.SyntaxError) syntax error at or near "```"\nLINE 1: ```sql\n        ^\n\n[SQL: ```sql\nSELECT COUNT(*) AS order_count FROM "order";\n```]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

In [39]:
chain.get_prompts()[0].pretty_print()


You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLR

In [40]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


answers_prompt = PromptTemplate.template(
   
  
  
  
)

AttributeError: template